In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
directory = 'Deteccion/RetinaFace/Friends'

In [107]:
frame_1 = np.load(f"{directory}/150/descriptores.npy")
frame_2 = np.load(f"{directory}/1000/descriptores.npy")

In [108]:
frame_1_norm = (frame_1.T / np.linalg.norm(frame_1, axis= 1)).T

In [109]:
frame_1_norm.shape

(4, 512)

In [110]:
frame_2_norm = (frame_2.T / np.linalg.norm(frame_2, axis= 1)).T

In [111]:
frame_2_norm.shape

(3, 512)

In [112]:
scores = frame_1_norm@frame_2_norm.T
scores

array([[ 0.00636635,  0.09058239,  0.41090506],
       [ 0.30299425, -0.19454922, -0.05732826],
       [-0.00478255, -0.05073496, -0.01780847],
       [ 0.07954722, -0.01819022, -0.11592321]], dtype=float32)

In [113]:
argmax = np.argmax(scores, axis=0) 
argmax

array([1, 0, 0])

In [114]:
np.max(scores, axis=0) 

array([0.30299425, 0.09058239, 0.41090506], dtype=float32)

In [115]:
ix = np.max(scores, axis=0) > 0.3
ix

array([ True, False,  True])

In [121]:
res = ix * argmax + ~ix * -np.ones(len(ix))
res 

array([ 1., -1.,  0.])

In [130]:
names = np.append(np.array([100, 200, 300, 400]), -1)
names[[1, -1, 0]]

array([200,  -1, 100])

In [203]:
class Personas:
    
    existentes = []
    path = ''
    thr = 0.3
    directory = 'Deteccion/RetinaFace/Friends'
    EMOCIONES = [
        "angry", 
        "disgust", 
        "scared", 
        "happy", 
        "sad", 
        "surprised", 
        "neutral"
    ]
    
    def __init__(self, descriptor, frame, index):
        self.descriptores = []
        self.data = pd.DataFrame(columns=[
            "index",
            "angry", 
            "disgust", 
            "scared", 
            "happy", 
            "sad", 
            "surprised", 
            "neutral",
            "pitch",
            "roll",
            "yaw"
        ])
        self.add(descriptor, frame, index)
        
    def add_descriptor(self, descriptor):
        normalizado = self.normalizar(descriptor)
        self.descriptores.append(normalizado)
    
    def add_emocion(self, frame, index):
        emociones = np.load(f'{directory}/{frame}/emociones.npy')
        self.data.loc[frame, self.EMOCIONES] = emociones[index]
    
    def add_headpose(self, frame, index):
        pass
    
    def add_bb(self, frame, index):
        pass
    
    def add(self, descriptor, frame, index):
        self.data.loc[frame, "index"] = index
        self.add_descriptor(descriptor)
        self.add_emocion(frame, index)
        self.add_headpose(frame, index)
        self.add_bb(frame, index)
    
    @staticmethod
    def normalizar(descriptor):
        if len(descriptor.shape) > 1:
            return (descriptor.T / np.linalg.norm(descriptor, axis= 1)).T
        else:
            return descriptor / np.linalg.norm(descriptor)
    
    @classmethod
    def juntar_personas(cls):
        total = []
        indices_list = []
        for i, persona in enumerate(cls.existentes):
            total.extend(persona.descriptores)
            indices_list.extend([i] * len(persona.descriptores))
        
        database = np.array(total)
        indices = np.array(indices_list)
        
        return database, indices
            
    
    @classmethod
    def encontrar_similar(cls, descriptores):
        """
        Recibe los descriptores de un nuevo frame.
        
        Si encuentra una persona similar entonces
        retorna el índice de la persona. 
        
        Sino retorna -1
        
        Si se recibe una matriz entonces el output
        es una lista
        """
        if len(descriptores)==0:
            return []
        
        # Juntar descriptores de personas existentes
        database, indices = cls.juntar_personas() 
        if len(database)==0:
            return [-1] * len(descriptores)
        
        # Comparar descriptores existentes con descriptores nuevos
        scores = database@descriptores.T
        
        # Encontrar indices con mayor score
        argmax = np.argmax(scores, axis=0) 
        
        # Encontrar indices donde el maximo supera el threshold
        ix = np.max(scores, axis=0) > cls.thr
        
        # Encontrar caras más parecidas
        sim_faces = ix * argmax + ~ix * -np.ones(len(ix))
        
        # Encontrar indices de personas
        indices = np.append(indices, -1)
        results = indices[sim_faces.astype(int)]
        
        return results
    
    @classmethod
    def nueva_persona(cls, descriptor, frame, index):
        """
        Agrega una nueva persona a lista de personas
        """
        cls.existentes.append(cls(descriptor, frame, index))
    
    @classmethod
    def agregar_descriptores(cls, identity, descriptores, frame):
        for index_origin, index_target in enumerate(identity):
            descriptor = descriptores[index_origin]
            if index_target >= 0:
                # Agregar descriptor a persona correspondiente
                persona = cls.existentes[index_target]
                persona.add(descriptor, frame, index_origin)
            else:
                # Agregar nueva persona
                cls.nueva_persona(descriptor, frame, index_origin)
    
    @classmethod
    def reset(cls):
        cls.existentes = []

In [204]:
Personas.reset()
for i in range(40):
    frame = np.load(f"{directory}/{i}/descriptores.npy")
    results = Personas.encontrar_similar(frame)
    Personas.agregar_descriptores(results, frame, i)

In [205]:
len(Personas.existentes[0].descriptores)

40

In [206]:
Personas.existentes[0].data

,index,angry,disgust,scared,happy,sad,surprised,neutral,pitch,roll,yaw
0,0,0.0605916,0.00439636,0.163676,0.105976,0.437814,0.00956166,0.217985,NaN,NaN,NaN
1,0,0.0358013,0.00336232,0.154518,0.0990854,0.434491,0.00920353,0.263538,NaN,NaN,NaN
2,0,0.0296305,0.00571881,0.143816,0.153038,0.269821,0.00970382,0.388271,NaN,NaN,NaN
3,0,0.0447929,0.0214209,0.132077,0.34365,0.189225,0.00770666,0.261127,NaN,NaN,NaN
4,0,0.0775742,0.0162405,0.167147,0.129144,0.315516,0.00700236,0.287376,NaN,NaN,NaN
5,0,0.0871559,0.0245417,0.188441,0.128384,0.32331,0.00712206,0.241046,NaN,NaN,NaN
6,0,0.0592054,0.02382,0.181719,0.186465,0.297571,0.00498295,0.246237,NaN,NaN,NaN
7,0,0.0826657,0.0225371,0.146828,0.100438,0.298938,0.00729222,0.341301,NaN,NaN,NaN
8,0,0.0886292,0.00262837,0.154497,0.0424788,0.410849,0.00698513,0.293933,NaN,NaN,NaN
9,0,0.0967668,0.00271104,0.1374,0.0456216,0.370011,0.00665116,0.340838,NaN,NaN,NaN


In [207]:
Personas.existentes[1].data

,index,angry,disgust,scared,happy,sad,surprised,neutral,pitch,roll,yaw
0,1,0.129491,0.00111534,0.0463445,0.0273935,0.0849974,0.0105302,0.700128,NaN,NaN,NaN
1,1,0.15072,0.00193867,0.0475138,0.0604405,0.0876748,0.0110678,0.640644,NaN,NaN,NaN
2,1,0.173361,0.00386459,0.0374429,0.0840561,0.0768111,0.00710518,0.617359,NaN,NaN,NaN
3,1,0.174754,0.00435174,0.0571104,0.0997265,0.0800052,0.0077135,0.576339,NaN,NaN,NaN
4,1,0.179711,0.00391713,0.0535906,0.0845384,0.0919304,0.00692739,0.579385,NaN,NaN,NaN
5,1,0.188261,0.00328831,0.0645917,0.0810587,0.107985,0.00837265,0.546443,NaN,NaN,NaN
6,1,0.194449,0.00565914,0.0682373,0.105163,0.097182,0.0137876,0.515523,NaN,NaN,NaN
7,1,0.165114,0.00310895,0.0551697,0.14672,0.0916012,0.0145362,0.52375,NaN,NaN,NaN
8,1,0.0925082,0.00777524,0.037213,0.24055,0.0777208,0.00836867,0.535864,NaN,NaN,NaN
9,1,0.112147,0.00217608,0.0599942,0.0836527,0.154739,0.0154758,0.571815,NaN,NaN,NaN


In [208]:
Personas.existentes[2].data

,index,angry,disgust,scared,happy,sad,surprised,neutral,pitch,roll,yaw
36,0,0.187983,0.0191112,0.230469,0.0933016,0.0716735,0.00687361,0.390588,NaN,NaN,NaN
37,0,0.172312,0.0151837,0.217673,0.18264,0.0577653,0.0232873,0.331138,NaN,NaN,NaN
38,0,0.164391,0.0139124,0.214681,0.135206,0.0596528,0.0119053,0.400252,NaN,NaN,NaN
39,0,0.159896,0.0164637,0.245906,0.0892377,0.064404,0.0104809,0.413612,NaN,NaN,NaN


In [209]:
Personas.existentes[3].data

,index,angry,disgust,scared,happy,sad,surprised,neutral,pitch,roll,yaw
36,1,0.0736896,0.00344158,0.164303,0.149441,0.0697518,0.0125832,0.52679,NaN,NaN,NaN


In [210]:
Personas.existentes[4].data

,index,angry,disgust,scared,happy,sad,surprised,neutral,pitch,roll,yaw
36,3,0.122779,0.00285459,0.148552,0.443255,0.111405,0.00483938,0.166315,NaN,NaN,NaN
37,3,0.145511,0.00576311,0.134728,0.387688,0.051198,0.0123571,0.262756,NaN,NaN,NaN
38,2,0.113774,0.00565414,0.13089,0.421278,0.0533344,0.0116675,0.263402,NaN,NaN,NaN
39,2,0.102576,0.00408696,0.123547,0.437483,0.0514536,0.0126731,0.268181,NaN,NaN,NaN


In [212]:
Personas.existentes[5].data

IndexError: list index out of range

In [211]:
a = pd.DataFrame(columns=[
            "index",
            "angry", 
            "disgust", 
            "scared", 
            "happy", 
            "sad", 
            "surprised", 
            "neutral",
            "pitch",
            "roll",
            "yaw"
        ])

In [195]:
a.loc[4, "angry"] = 1
a.loc[10, "angry"] = 1

a

,index,angry,disgust,scared,happy,sad,surprised,neutral,pitch,roll,yaw
4,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [196]:
b = a.copy()

In [197]:
b.reset_index().rename(columns={'level_0': 'frame'})

,frame,index,angry,disgust,scared,happy,sad,surprised,neutral,pitch,roll,yaw
0,4,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [199]:
b.loc[:, 'id'] = 'persona_1'

In [200]:
b

,index,angry,disgust,scared,happy,sad,surprised,neutral,pitch,roll,yaw,id
4,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,persona_1
10,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,persona_1
